In [1]:
from backtest import ticker_stats, smooth, features, stringify, print_ticker_heading, balanced_scorecard, \
                     determine_minima_n_maxima, align_minima_n_maxima, plot_trades
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score
from sklearn.datasets import load_breast_cancer, load_iris, make_moons, make_circles, make_classification
from sklearn.linear_model import LogisticRegression
from category_encoders import WOEEncoder

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler, KBinsDiscretizer, FunctionTransformer
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer

import matplotlib.pyplot as plt
%matplotlib inline

from scipy.signal import savgol_filter, argrelmin, argrelmax

import gc; gc.enable()

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
DATAPATH = '/Users/frkornet/Flatiron/Stock-Market-Final-Project/data/'
sdf = pd.read_csv(f'{DATAPATH}optimal_params.csv')

sdf = sdf.loc[sdf.up3_tpr > 0.0].reset_index()
if 'index' in sdf.columns:
    del sdf['index']
sdf.head(20)

,NAME_OF_ISSUER,CUSIP,VALUE,SHRS_OR_PRN_AMT,SOLE_VOTING_AUTH,NO_VOTING_AUTH,TICKER,up3_period,up3_lags,up3_tpr,...,up8_lags,up8_tpr,up10_period,up10_lags,up10_tpr,naive1_tpr,naive3_tpr,naive5_tpr,naive10_tpr,naive8_tpr
0,ADURO BIOTECH INC,00739L101,2902,2738200,2590838,147362,ADRO,15y,"1,3",0.9452,...,"1,3,5,10,20,30",0.9873,3y,"1,3,5",0.9595,0,0.4286,0.4091,0.4286,0.4740
1,BMC STK HLDGS INC,05591B109,34168,1305100,1175955,129145,BMCH,3y,"1,3",0.9878,...,"1,3,5",1.0000,3y,"1,3",1.0000,0,0.5519,0.5584,0.6104,0.6104
2,CVR ENERGY INC,12662P108,92673,2104769,2104769,0,CVI,8y,"1,3",0.8873,...,"1,3,5,10,20,30,45",1.0000,5y,"1,3",0.9831,0,0.4026,0.3831,0.3117,0.3636
3,GRACO INC,384109104,8329,180900,72895,108005,GGG,5y,"1,3",0.9053,...,"1,3,5,10,20",1.0000,15y,"1,3,5",0.9898,0,0.5519,0.6104,0.5649,0.5584
4,GULF RESOURCES INC,40251W309,1276,1830162,1828362,1800,GURE,5y,"1,3",0.7941,...,"1,3",0.9615,15y,"1,3",1.0000,0,0.3377,0.3182,0.2857,0.2792
5,HANCOCK WHITNEY CORPORATION,410120109,9687,252962,252962,0,HWC,5y,"1,3",0.8846,...,"1,3,5,10,20,30,45",0.9880,3y,"1,3",1.0000,0,0.4870,0.4870,0.4545,0.4740
6,MBIA INC,55262C100,2108,228374,146742,81632,MBI,8y,"1,3",0.8824,...,"1,3",1.0000,3y,"1,3",0.9865,0,0.4091,0.4286,0.3831,0.4026
7,PALO ALTO NETWORKS INC,697435105,1192028,5848150,5622490,225660,PANW,3y,"1,3,5,10",0.9495,...,"1,3",0.9908,5y,"1,3",1.0000,0,0.6234,0.6494,0.5974,0.6299
8,SI BONE INC,825704109,723,40900,40900,0,SIBN,15y,"1,3,5,10,20,30,45,60",0.8933,...,"1,3",0.9684,15y,"1,3",0.9775,0,0.4935,0.5260,0.5130,0.5519
9,SPECTRUM PHARMACEUTICALS INC,84763A108,22936,2765017,2747310,17707,SPPI,3y,"1,3,5,10",0.9125,...,"1,3,5,10",1.0000,10y,"1,3,5",0.9722,0,0.4675,0.4351,0.3831,0.3961


In [4]:
tickers = sdf.TICKER.to_list()
print(tickers)

['ADRO', 'BMCH', 'CVI', 'GGG', 'GURE', 'HWC', 'MBI', 'PANW', 'SIBN', 'SPPI']


In [5]:
min_indices, max_indices = determine_minima_n_maxima(tickers, False)

tickers= ['ADRO', 'BMCH', 'CVI', 'GGG', 'GURE', 'HWC', 'MBI', 'PANW', 'SIBN', 'SPPI']


In [6]:
min_indices, max_indices = align_minima_n_maxima(tickers, min_indices, max_indices, True)

Ticker: ADRO min_id= 23 max_id= 42
      ADRO min_id= 23 max_id= 42 
Ticker: BMCH min_id= 26 max_id= 105
      BMCH min_id= 26 max_id= 105 
Ticker: CVI min_id= 92 max_id= 21
      CVI min_id= 92 max_id= 115  (*)
Ticker: GGG min_id= 88 max_id= 50
      GGG min_id= 88 max_id= 111  (*)
Ticker: GURE min_id= 109 max_id= 24
      GURE min_id= 109 max_id= 115  (*)
Ticker: HWC min_id= 133 max_id= 29
      HWC min_id= 133 max_id= 216  (*)
Ticker: MBI min_id= 84 max_id= 48
      MBI min_id= 84 max_id= 172  (*)
Ticker: PANW min_id= 106 max_id= 36
      PANW min_id= 106 max_id= 152  (*)
Ticker: SIBN min_id= 22 max_id= 53
      SIBN min_id= 22 max_id= 53 
Ticker: SPPI min_id= 87 max_id= 7
      SPPI min_id= 87 max_id= 113  (*)


In [8]:
# suppress output of plotting all the trades as this is quite a lot...
# plot_trades(tickers, min_indices, max_indices)